In [1]:
import pandas as pd

# Establecer la ruta de trabajo y cargar el archivo CSV
exp_path = "outputs/exp5_aleatory_linear_circle_3_9_1000/"
indicators_file = exp_path + 'data.csv'
df = pd.read_csv(indicators_file)
df = df[df['n_local_networks'] == 9]
df

,i_sample,n_local_networks,n_var_network,v_topology,n_output_variables,n_clauses_function,n_local_attractors,n_pair_attractors,n_attractor_fields,n_time_find_attractors,n_time_find_pairs,n_time_find_fields
6,1,9,5,4,2,2,17,15,1,0.483123,0.000201,0.000099
13,1,9,5,3,2,2,18,18,1,0.591002,0.000180,0.000133
20,1,9,5,2,2,2,19,21,1,0.618477,0.000250,0.000353
27,2,9,5,4,2,2,26,46,512,0.801684,0.000264,0.015149
34,2,9,5,3,2,2,27,54,512,0.823553,0.000208,0.027018
...,...,...,...,...,...,...,...,...,...,...,...,...
20971,999,9,5,3,2,2,27,54,512,1.215548,0.000271,0.028531
20978,999,9,5,2,2,2,33,98,512,1.507390,0.000399,0.037034
20985,1000,9,5,4,2,2,35,93,19683,1.026523,0.000210,0.326266
20992,1000,9,5,3,2,2,36,108,19683,1.044213,0.000483,1.225516


In [2]:
import os
import pickle
import pandas as pd

# Define el número de muestras y topologías
n_samples = 100
l_topologies = [2, 3, 4]

# Ruta base para los archivos pickle
base_path_cbn = f'{exp_path}pkl_cbn/cbn'
dict_global_scenes = {}

# Cargar los conteos de escenas globales
for i_sample in range(n_samples + 1):
    for i_topology in l_topologies:
        base_path_cbn_sample_topology = f'{base_path_cbn}_{i_sample}_{i_topology}.pkl'
        
        if os.path.exists(base_path_cbn_sample_topology):
            with open(base_path_cbn_sample_topology, 'rb') as f:
                o_cbn = pickle.load(f)
                o_cbn.count_fields_by_global_scenes()  # Asegúrate de que esta función esté definida en tu objeto
            dict_global_scenes[(i_sample, i_topology)] = o_cbn.d_global_scenes_count  # Asegúrate de que esta propiedad sea accesible

# Convertir el diccionario a DataFrame
df_global_scenes = pd.DataFrame.from_dict(dict_global_scenes, orient='index').fillna(0)

# Calcular columnas
df_global_scenes['stable_global_scenes'] = (df_global_scenes != 0).sum(axis=1)
df_global_scenes['total_global_scenes'] = df_global_scenes.index.get_level_values(1).map({2: 2**8, 3: 2**9, 4: 2**8})
df_global_scenes['no_stable_global_scenes'] = df_global_scenes['total_global_scenes'] - df_global_scenes['stable_global_scenes']
df_global_scenes['perc_stable_global_scenes'] = (df_global_scenes['stable_global_scenes'] / df_global_scenes['total_global_scenes']) * 100
df_global_scenes['perc_no_stable_global_scenes'] = (df_global_scenes['no_stable_global_scenes'] / df_global_scenes['total_global_scenes']) * 100

# Agrupar por topología y calcular la media
df_global_scenes_grouped = df_global_scenes.groupby(df_global_scenes.index.get_level_values(1)).mean()

# Mostrar el DataFrame agrupado
df_global_scenes_grouped

,111111111,11111111,11101001111,11101111110,11101111111,11100001111,11111001111,11111111110,11111111111,11110001111,...,00101011,00101010,0011111111,1000110011,0111001100,stable_global_scenes,total_global_scenes,no_stable_global_scenes,perc_stable_global_scenes,perc_no_stable_global_scenes
2,4059.406593,0.000000,0.021978,0.043956,0.043956,0.725275,2.153846,4.263736,931.516484,0.043956,...,0.000000,0.00000,0.010989,0.175824,0.351648,5.802198,256.0,250.197802,2.266484,97.733516
3,72550.282609,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.000000,0.000000,0.000000,10.097826,512.0,501.902174,1.972232,98.027768
4,0.000000,73847.857143,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,2.967033,1.78022,0.000000,0.000000,0.000000,6.725275,256.0,249.274725,2.627060,97.372940


In [4]:
print (df_global_scenes_grouped[['total_global_scenes','stable_global_scenes','no_stable_global_scenes','perc_stable_global_scenes','perc_no_stable_global_scenes']])

   total_global_scenes  stable_global_scenes  no_stable_global_scenes  \
2                256.0              5.802198               250.197802   
3                512.0             10.097826               501.902174   
4                256.0              6.725275               249.274725   

   perc_stable_global_scenes  perc_no_stable_global_scenes  
2                   2.266484                     97.733516  
3                   1.972232                     98.027768  
4                   2.627060                     97.372940  


In [5]:
# Diccionario para mapear los valores de la topología a los nombres correspondientes
topology_mapping = {2: 'Aleatory', 3: 'Circular', 4: 'Linear'}

# Comenzar la construcción de la tabla en LaTeX
latex_code = r'\begin{table}[ht]' + '\n'
latex_code += r'\centering' + '\n'
latex_code += r'\begin{tabular}{|c|c|c|c|c|c|}' + '\n'
latex_code += r'\hline' + '\n'
latex_code += r'\textbf{Topology} & \textbf{Total Global Scenes} & \textbf{Stable Global Scenes} & \textbf{No Stable Global Scenes} & \textbf{\% Stable Global Scenes} & \textbf{\% No Stable Global Scenes} \\ \hline' + '\n'

# Iterar sobre el DataFrame para extraer las filas
for index, row in df_global_scenes_grouped.iterrows():
    topology = topology_mapping.get(index, 'Unknown')  # Obtener el nombre de la topología
    total_global_scenes = f"{row['total_global_scenes']:.1f}"
    stable_global_scenes = f"{row['stable_global_scenes']:.2f}"
    no_stable_global_scenes = f"{row['no_stable_global_scenes']:.2f}"
    perc_stable_global_scenes = f"{row['perc_stable_global_scenes']:.2f}\\%"
    perc_no_stable_global_scenes = f"{row['perc_no_stable_global_scenes']:.2f}\\%"
    
    # Agregar la fila a la tabla
    latex_code += f"{topology} & {total_global_scenes} & {stable_global_scenes} & {no_stable_global_scenes} & {perc_stable_global_scenes} & {perc_no_stable_global_scenes} \\\\ \\hline\n"

# Finalizar la tabla
latex_code += r'\end{tabular}' + '\n'
latex_code += r'\caption{Summary of Global Scenes by Topology}' + '\n'
latex_code += r'\label{tab:global_scenes}' + '\n'
latex_code += r'\end{table}'

# Mostrar el código LaTeX generado
print(latex_code)


\begin{table}[ht]
\centering
\begin{tabular}{|c|c|c|c|c|c|}
\hline
\textbf{Topology} & \textbf{Total Global Scenes} & \textbf{Stable Global Scenes} & \textbf{No Stable Global Scenes} & \textbf{\% Stable Global Scenes} & \textbf{\% No Stable Global Scenes} \\ \hline
Aleatory & 256.0 & 5.80 & 250.20 & 2.27\% & 97.73\% \\ \hline
Circular & 512.0 & 10.10 & 501.90 & 1.97\% & 98.03\% \\ \hline
Linear & 256.0 & 6.73 & 249.27 & 2.63\% & 97.37\% \\ \hline
\end{tabular}
\caption{Summary of Global Scenes by Topology}
\label{tab:global_scenes}
\end{table}
